In [1]:
!pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

In [3]:
spark=SparkSession.builder.getOrCreate()

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
import pandas as pd
data=spark.read.csv('drive/My Drive/ABD-DataFrame/indian_food.csv', header=True, inferSchema=True)
data.show(truncate=False)

+--------------+----------------------------------------------------------------------------------------------+----------+---------+---------+--------------+-------+-------------+
|name          |ingredients                                                                                   |diet      |prep_time|cook_time|flavor_profile|course |state        |
+--------------+----------------------------------------------------------------------------------------------+----------+---------+---------+--------------+-------+-------------+
|Balu shahi    |Maida flour, yogurt, oil, sugar                                                               |vegetarian|45       |25       |sweet         |dessert|West Bengal  |
|Boondi        |Gram flour, ghee, sugar                                                                       |vegetarian|80       |30       |sweet         |dessert|Rajasthan    |
|Gajar ka halwa|Carrots, milk, sugar, ghee, cashews, raisins                                        

In [7]:
data.printSchema()

root
 |-- name: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- diet: string (nullable = true)
 |-- prep_time: integer (nullable = true)
 |-- cook_time: integer (nullable = true)
 |-- flavor_profile: string (nullable = true)
 |-- course: string (nullable = true)
 |-- state: string (nullable = true)



In [8]:
# 1. Find out how many unique dishes are present.

data.select('name').distinct().count()
data.select('name').distinct().show()


+-----------------+
|             name|
+-----------------+
|            Kheer|
|          Poriyal|
|Sabudana Khichadi|
|   Gajar ka halwa|
|     Keerai sadam|
|         Dalithoy|
|         Mihidana|
|          Singori|
|          Uttapam|
|          Gavvalu|
|   Kakinada khaja|
|   Chak Hao Kheer|
|          Pachadi|
|         Vindaloo|
|     Gheela Pitha|
|         Idiappam|
|     Keerai kootu|
|             Idli|
|            Saath|
|            Rabri|
+-----------------+
only showing top 20 rows



In [9]:
# 2. Which state has more dishes?
data.select('state','name').groupBy('state').count().orderBy(col('count').desc()).show()
data.select('state','name').groupBy('state').count().orderBy(col('count').desc()).show(1)

+---------------+-----+
|          state|count|
+---------------+-----+
|        Gujarat|   35|
|         Punjab|   32|
|    Maharashtra|   30|
|             -1|   24|
|    West Bengal|   24|
|          Assam|   21|
|     Tamil Nadu|   20|
| Andhra Pradesh|   10|
|  Uttar Pradesh|    9|
|         Kerala|    8|
|         Odisha|    7|
|      Karnataka|    6|
|      Rajasthan|    6|
|      Telangana|    5|
|            Goa|    3|
|          Bihar|    3|
| Madhya Pradesh|    2|
|        Manipur|    2|
|Jammu & Kashmir|    2|
|       Nagaland|    1|
+---------------+-----+
only showing top 20 rows

+-------+-----+
|  state|count|
+-------+-----+
|Gujarat|   35|
+-------+-----+
only showing top 1 row



In [10]:
# 3. How many dishes from state Karnataka?
data.select('name','state').where(col('state')=='Karnataka').show()
data.select('name','state').where(col('state')=='Karnataka').count()

+--------------+---------+
|          name|    state|
+--------------+---------+
| Dharwad pedha|Karnataka|
|    Mysore pak|Karnataka|
|Obbattu holige|Karnataka|
|Bisi bele bath|Karnataka|
|     Koshambri|Karnataka|
|       Sandige|Karnataka|
+--------------+---------+



6

In [11]:
#4. List number of unique regions
def mapRegion(state):
    south = ['Tamil Nadu', 'Karnataka', 'Kerala', 'Goa', 'Andhra Pradesh']
    west = ['Rajasthan', 'Gujarat', 'Maharashtra']
    east = ['West Bengal', 'Assam']
    north = ['Punjab', 'NCT of Delhi', 'Haryana']
    central = ['Madhya Pradesh', 'Chhattisgarh']

    if state in south:
        return 'South'
    elif state in north:
        return 'North'
    elif state in west:
        return 'West'
    elif state in east:
        return 'East'
    elif state in central:
        return 'Central'
    else:
        return '-1'

regionudf = udf(mapRegion, StringType())
df1 = data.withColumn('Region', regionudf(col('state')))
df1.show()

+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|          name|         ingredients|      diet|prep_time|cook_time|flavor_profile| course|        state|Region|
+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|    Balu shahi|Maida flour, yogu...|vegetarian|       45|       25|         sweet|dessert|  West Bengal|  East|
|        Boondi|Gram flour, ghee,...|vegetarian|       80|       30|         sweet|dessert|    Rajasthan|  West|
|Gajar ka halwa|Carrots, milk, su...|vegetarian|       15|       60|         sweet|dessert|       Punjab| North|
|        Ghevar|Flour, ghee, kewr...|vegetarian|       15|       30|         sweet|dessert|    Rajasthan|  West|
|   Gulab jamun|Milk powder, plai...|vegetarian|       15|       40|         sweet|dessert|  West Bengal|  East|
|        Imarti|Sugar syrup, lent...|vegetarian|       10|       50|         sweet|dessert|  Wes

In [12]:
res=df1.select('Region').distinct().count()
df1.select('Region').distinct().show()
print(res)

+-------+
| Region|
+-------+
|     -1|
|  South|
|Central|
|   East|
|   West|
|  North|
+-------+

6


In [13]:
# 5. Count number of dishes from each region
df1.select('region').groupBy('region').count().show()


# (OR)
df1.select('region').groupBy('region').count().withColumn('dishes',row_number().over(Window.partitionBy('region').orderBy(desc('count')))).filter(col('dishes') == 1).show()

+-------+-----+
| region|count|
+-------+-----+
|     -1|   55|
|  South|   47|
|Central|    3|
|   East|   45|
|   West|   71|
|  North|   34|
+-------+-----+

+-------+-----+------+
| region|count|dishes|
+-------+-----+------+
|     -1|   55|     1|
|Central|    3|     1|
|   East|   45|     1|
|  North|   34|     1|
|  South|   47|     1|
|   West|   71|     1|
+-------+-----+------+



In [14]:
# 6. List unique 'flavor_profile' and 'course'

df1.select('flavor_profile','course').distinct().show()

+--------------+-----------+
|flavor_profile|     course|
+--------------+-----------+
|        bitter|      snack|
|         spicy|    starter|
|          sour|main course|
|            -1|      snack|
|            -1|main course|
|         sweet|main course|
|        bitter|main course|
|         spicy|      snack|
|         sweet|    dessert|
|         spicy|main course|
+--------------+-----------+



In [15]:
# 7. Which state has more 'main course'?
#df1.select('course').filter(col('course')=='main course').show()
df1.select('state','course').filter(col('course')=='main course').groupBy(col('state')).count().orderBy(col('count').desc()).show()
df1.select('state').filter(col('course')=='main course').groupBy(col('state')).count().orderBy(col('count').desc()).show(1)

+---------------+-----+
|          state|count|
+---------------+-----+
|         Punjab|   28|
|     Tamil Nadu|   17|
|          Assam|   15|
|        Gujarat|   12|
|    Maharashtra|   12|
|             -1|    9|
|    West Bengal|    9|
|         Kerala|    5|
|      Karnataka|    4|
|      Rajasthan|    3|
|  Uttar Pradesh|    3|
|          Bihar|    2|
|       Nagaland|    1|
|         Odisha|    1|
| Madhya Pradesh|    1|
|        Manipur|    1|
|Jammu & Kashmir|    1|
|            Goa|    1|
|        Haryana|    1|
|   NCT of Delhi|    1|
+---------------+-----+
only showing top 20 rows

+------+-----+
| state|count|
+------+-----+
|Punjab|   28|
+------+-----+
only showing top 1 row



In [16]:
# 8. Give the %of dishes from each region.
total=df1.count()
df1.select('region').groupBy('region').count().withColumn('Percentage',(col('count')/total)*100).show()


+-------+-----+------------------+
| region|count|        Percentage|
+-------+-----+------------------+
|     -1|   55|21.568627450980394|
|  South|   47| 18.43137254901961|
|Central|    3|1.1764705882352942|
|   East|   45|17.647058823529413|
|   West|   71| 27.84313725490196|
|  North|   34|13.333333333333334|
+-------+-----+------------------+



In [17]:
# 9. List the states which has more dishes from each region.
df1.select('state','region').groupBy('state','region').count().withColumn('dishes',row_number().over(Window.partitionBy('region').orderBy(desc('count')))).filter(col('dishes') == 1).show()

+--------------+-------+-----+------+
|         state| region|count|dishes|
+--------------+-------+-----+------+
|            -1|     -1|   24|     1|
|Madhya Pradesh|Central|    2|     1|
|   West Bengal|   East|   24|     1|
|        Punjab|  North|   32|     1|
|    Tamil Nadu|  South|   20|     1|
|       Gujarat|   West|   35|     1|
+--------------+-------+-----+------+

